In [1]:
# Povm

In [1]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.data_analysis.projected_gradient_descent_backtracking import ProjectedGradientDescentBacktracking, ProjectedGradientDescentBacktrackingOption
from quara.data_analysis.weighted_probability_based_squared_error import WeightedProbabilityBasedSquaredError, WeightedProbabilityBasedSquaredErrorOption
from quara.data_analysis.weighted_relative_entropy import WeightedRelativeEntropy, WeightedRelativeEntropyOption
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.state import (
    State,
    get_x0_1q,
    get_x1_1q,
    get_y0_1q,
    get_y1_1q,
    get_z0_1q,
    get_z1_1q,
)
from quara.objects.povm import (
    Povm,get_x_povm
)
from quara.objects.qoperation import QOperation
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import LossMinimizationEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

from quara.data_analysis.simulation import StandardQTomographySimulationSetting

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
start_all = time.time()

In [6]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

# |+><+|
state_x0 = get_x0_1q(c_sys)
# |+i><+i|
state_y0 = get_y0_1q(c_sys)
# |0><0|
state_z0 = get_z0_1q(c_sys)
# |1><1|
state_z1 = get_z1_1q(c_sys)
tester_objects = [state_x0, state_y0, state_z0, state_z1]

In [7]:
# Case 1: 軸上
# a0, a1, a2, a3 = 1, 1, 0, 0
# m1 = (1 / np.sqrt(2)) * np.array([a0, a1, a2, a3])
# m2 = (1 / np.sqrt(2)) * np.array([2 - a0, -a1, -a2, -a3])

# Case 2: 軸から離れた表面
a0, a1, a2, a3 = 1, 1 / np.sqrt(2), 0, 1 / np.sqrt(2)
m1 = (1 / np.sqrt(2)) * np.array([a0, a1, a2, a3])
m2 = (1 / np.sqrt(2)) * np.array([2 - a0, -a1, -a2, -a3])

# Case 3: 原点
# a0, a1, a2, a3 = 1, 0, 0, 0
# m1 = (1 / np.sqrt(2)) * np.array([a0, a1, a2, a3])
# m2 = (1 / np.sqrt(2)) * np.array([2 - a0, -a1, -a2, -a3])

true_object = Povm(vecs=[m1, m2], c_sys=c_sys)

true_object.vecs

(array([0.70710678, 0.5       , 0.        , 0.5       ]),
 array([ 0.70710678, -0.5       ,  0.        , -0.5       ]))

In [25]:
num_data = [100, 1000]
n_rep = 100
measurement_n = len(true_object.vecs)  # 測定値の数

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
    "Maximum-Likelihood(True)",
    "Maximum-Likelihood(False)",
    "Least Squares(True)",
    "Least Squares(False)",
]

seed = 777
qtomography_list = [
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=True, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=False, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=True, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=False, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=True, eps_proj_physical=1e-13, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=False, eps_proj_physical=1e-13, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=True, eps_proj_physical=1e-13, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=False, eps_proj_physical=1e-13, seed=seed),
]

para_list = [
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
]
order = "eq_ineq"
estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(mode_proj_order=order),
    ProjectedLinearEstimator(mode_proj_order=order),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
]

loss_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropy(4),
    WeightedRelativeEntropy(8),
    WeightedProbabilityBasedSquaredError(4),
    WeightedProbabilityBasedSquaredError(8),
]

loss_option_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropyOption("identity"),
    WeightedRelativeEntropyOption("identity"),
    WeightedProbabilityBasedSquaredErrorOption("identity"),
    WeightedProbabilityBasedSquaredErrorOption("identity"),
]

algo_list = [
    None,
    None,
    None,
    None,
    ProjectedGradientDescentBacktracking(),
    ProjectedGradientDescentBacktracking(),
    ProjectedGradientDescentBacktracking(),
    ProjectedGradientDescentBacktracking(),
]

algo_option_list = [
    None,
    None,
    None,
    None,
    
    ProjectedGradientDescentBacktrackingOption(
        mode_stopping_criterion_gradient_descent="sum_absolute_difference_variable",
        num_history_stopping_criterion_gradient_descent=1,
        mode_proj_order=order
    ),
        ProjectedGradientDescentBacktrackingOption(
        mode_stopping_criterion_gradient_descent="sum_absolute_difference_variable",
        num_history_stopping_criterion_gradient_descent=1,
        mode_proj_order=order
    ),
        ProjectedGradientDescentBacktrackingOption(
        mode_stopping_criterion_gradient_descent="sum_absolute_difference_variable",
        num_history_stopping_criterion_gradient_descent=1,
        mode_proj_order=order
    ),
        ProjectedGradientDescentBacktrackingOption(
        mode_stopping_criterion_gradient_descent="sum_absolute_difference_variable",
        num_history_stopping_criterion_gradient_descent=1,
        mode_proj_order=order
    )
]

estimation_results_list = []
elapsed_times = []
simulation_settings = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]
    loss = loss_list[i]
    loss_option = loss_option_list[i]
    algo = algo_list[i]
    algo_option = algo_option_list[i]
    
    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
       qtomography=qtomography,
       true_object=true_object,
       num_data=num_data,
       estimator=estimator,
       loss=loss,
       loss_option=loss_option,
       algo=algo,
       algo_option=algo_option,
       iteration=n_rep,
    )
    estimation_results_list.append(estimation_results)
    
    # stock settings of this simulation
    simulation_setting = StandardQTomographySimulationSetting(
        name=name,
        estimator=estimator,
        loss=loss,
        loss_option=loss_option,
        algo=algo,
        algo_option=algo_option,
        true_object = true_object,
        tester_objects = tester_objects,
        seed=seed,
        n_rep=n_rep
    )
    
    simulation_settings.append(simulation_setting)

    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")
    elapsed_times.append(elapsed_time)

  0%|          | 0/100 [00:00<?, ?it/s]

Case 0: LinearEstimator(True)
Parametorization: True
Type of qtomography: StandardPovmt
Estimator: LinearEstimator


  4%|▍         | 4/100 [00:00<00:02, 34.08it/s]

elapsed_time:0.06045491298039754[min]

Case 1: LinearEstimator(False)
Parametorization: False
Type of qtomography: StandardPovmt
Estimator: LinearEstimator


  0%|          | 0/100 [00:00<?, ?it/s]

...

elapsed_time:6.592624584833781[min]

Case 6: Least Squares(True)
Parametorization: True
Type of qtomography: StandardPovmt
Estimator: LossMinimizationEstimator


  0%|          | 0/100 [00:00<?, ?it/s]

elapsed_time:5.850670830408732[min]

Case 7: Least Squares(False)
Parametorization: False
Type of qtomography: StandardPovmt
Estimator: LossMinimizationEstimator


100%|██████████| 100/100 [24:35<00:00, 14.75s/it]  


elapsed_time:24.58725478251775[min]



In [10]:
"""
data_analysis.make_mses_graph_estimation_results(
    estimation_results_list,
    case_name_list,
    true_object,
    estimator_list=estimator_list,
)
"""

'\ndata_analysis.make_mses_graph_estimation_results(\n    estimation_results_list,\n    case_name_list,\n    true_object,\n    estimator_list=estimator_list,\n)\n'

In [27]:
report.export_report(f"20210226_povmt_nrep={n_rep}_軸上から離れた表面の少し内側_mode_proj_order={order}.pdf",
    estimation_results_list=estimation_results_list,
    simulation_settings=simulation_settings,
    seed=seed
)

​Generating table of computation time ...
​Generating table of tolerance of physicality constraint violation ...
​Generating table of experimental conditions ...
Generating case list ...
Computation time of estimators ...


100%|██████████| 100/100 [00:00<00:00, 117093.91it/s]

​​Generating MSE of empirical distributions blocks ...


​​Generating consictency test blocks ...
​Generating a graph for MSE ...
  LinearEstimator(True)=[0.020663203166004503, 0.002471635556674566]
  LinearEstimator(False)=[0.025023630290750703, 0.0025565735748293954]
  ProjectedLinearEstimator(True)=[0.02173568415679148, 0.002109955343928706]
  ProjectedLinearEstimator(False)=[0.020083936388971393, 0.0022808904009212357]
  Maximum-Likelihood(True)=[0.01695788270519532, 0.0018915655456205487]
  Maximum-Likelihood(False)=[0.01827768883556187, 0.002150224472505702]
  Least Squares(True)=[0.01930044598756442, 0.0019148843932044388]
  Least Squares(False)=[0.021624433133689, 0.002315957491719625]
  LinearEstimator(True)=[0.020663203166004503, 0.002471635556674566]
  ProjectedLinearEstimator(True)=[0.02173568415679148, 0.002109955343928706]
  Maximum-Likelihood(True)=[0.01695788270519532, 0.0018915655456205487]
  Least Squares(True)=[0.01930044598756442, 0.0019148843932044388]
  LinearEstimator(False)=[0.025023630290750703, 0.0025565735748293954

100%|██████████| 100/100 [00:00<00:00, 24582.72it/s]

​​Generating physicality violation test blocks ...



  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 62.10it/s]

2it [00:00, 55.69it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 56.08it/s]

2it [00:00, 55.44it/s]
100%|██████████| 100/100 [00:00<00:00, 3200.71it/s]

type(source[0])=<class 'quara.objects.povm.Povm'>


TypeError: `source` must be list of EstimationResult or QOperation, not list of <class 'list'>

In [20]:
from quara.data_analysis.simulation_check import StandardQTomographySimulationCheck

results = []
for i in range(len(simulation_settings)):
    sim_check = StandardQTomographySimulationCheck(simulation_settings[i], estimation_results_list[i])
    print(f"Case {i}: {simulation_settings[i].name}")
    result = sim_check.execute_physicality_violation_check()
    results.append(result)
    print("-==============================")
    print()

Case 0: LinearEstimator(True)
[OK] N=100 is_eq_constraint_satisfied_all
True=100, False=0, eps=1e-13
[OK] N=1000 is_eq_constraint_satisfied_all
True=100, False=0, eps=1e-13
-==============================

Case 1: LinearEstimator(False)
[Skipped] Physicality Violation Check 
If on_para_eq_constraint is False in Linear Estimator, nothing is checked.
-==============================

Case 2: ProjectedLinearEstimator(True)
[NG] N=100 physicality violation check
True=38, False=62
[NG] N=1000 physicality violation check
True=37, False=63
-==============================

Case 3: ProjectedLinearEstimator(False)
[NG] N=100 physicality violation check
True=23, False=77
[NG] N=1000 physicality violation check
True=26, False=74
-==============================

Case 4: Maximum-Likelihood(True)
[OK] N=100 is_eq_constraint_satisfied_all
True=100, False=0, eps=1e-13
[OK] N=1000 is_eq_constraint_satisfied_all
True=100, False=0, eps=1e-13
[NG] N=100 is_ineq_constraint_satisfied_all
True=42, False=58, ep

In [14]:
from quara.data_analysis.simulation_check import StandardQTomographySimulationCheck

results = []
for i in range(len(simulation_settings)):
    sim_check = StandardQTomographySimulationCheck(simulation_settings[i], estimation_results_list[i])
    print(f"Case {i}: {simulation_settings[i].name}")
    result = sim_check.execute_all()
    results.append(result)
    print("-==============================")
    print()

100%|██████████| 2/2 [00:00<00:00, 35.01it/s]

Case 0: LinearEstimator(True)
[OK] N=100
|MSE_EmpiDist - MSE_Analytical| = 0.004839969244287908
eps = 0.010137034228419863
[NG] N=1000
|MSE_EmpiDist - MSE_Analytical| = 0.0007989543548555717
eps = 0.0003511537043399281
[OK] Consistency Check
result=9.599797807798177e-31
eps=1e-10
result < eps: True
[OK] N=100
|MSE_Linear - MSE_Analytical| = 0.0002411229460638903
eps = 0.07312831073421351
[OK] N=1000
|MSE_Linear - MSE_Analytical| = 0.0013397515569817456
eps = 0.0034742762128161238
[OK] N=100 is_eq_constraint_satisfied_all
True=2, False=0, eps=1e-13
[OK] N=1000 is_eq_constraint_satisfied_all
True=2, False=0, eps=1e-13
========== Summary ============
Name: LinearEstimator(True)
MSE of Empirical Distributions: NG
Consistency: OK
MSE of estimators: OK
Physicality Violation: OK
-==============================

Case 1: LinearEstimator(False)
[OK] N=100
|MSE_EmpiDist - MSE_Analytical| = 0.004166376115480755
eps = 0.020610173055266433
[OK] N=1000
|MSE_EmpiDist - MSE_Analytical| = 0.000341337146


100%|██████████| 2/2 [00:00<00:00, 3611.11it/s]


[OK] N=100
|MSE_EmpiDist - MSE_Analytical| = 0.0017889629438466031
eps = 0.021493290600952022
[OK] N=1000
|MSE_EmpiDist - MSE_Analytical| = 0.00022175414412919144
eps = 0.00202814758127016
[OK] Consistency Check
result=1.0563725744963989e-31
eps=1e-10
result < eps: True
[NG] N=100
MSE_Linear - MSE_ProjectedLinearEstimator = -0.0009644991818551496
[OK] N=1000
MSE_Linear - MSE_ProjectedLinearEstimator = 0.0028670770285831315
[NG] N=100 physicality violation check
True=0, False=2
[NG] N=1000 physicality violation check
True=0, False=2
========== Summary ============
Name: ProjectedLinearEstimator(False)
MSE of Empirical Distributions: OK
Consistency: OK
MSE of estimators: NG
Physicality Violation: NG
-==============================

Case 4: Maximum-Likelihood(True)
[OK] N=100
|MSE_EmpiDist - MSE_Analytical| = 0.0023242404917498093
eps = 0.03472292218890234
[OK] N=1000
|MSE_EmpiDist - MSE_Analytical| = 0.00033595435485557186
eps = 0.001119647071467588


100%|██████████| 2/2 [00:00<00:00, 597.01it/s]

...


100%|██████████| 2/2 [00:00<00:00, 5729.92it/s]

[OK] Consistency Check
result=9.103765623288926e-15
eps=1e-10
result < eps: True
[OK] N=100
MSE_Linear - MSE_LossMinimizationEstimator = 0.0169383897472777
[OK] N=1000
MSE_Linear - MSE_LossMinimizationEstimator = 0.0034478129331349046
[OK] N=100 is_eq_constraint_satisfied_all
True=2, False=0, eps=1e-13
[OK] N=1000 is_eq_constraint_satisfied_all
True=2, False=0, eps=1e-13
[NG] N=100 is_ineq_constraint_satisfied_all
True=0, False=2, eps=1e-09
[NG] N=1000 is_ineq_constraint_satisfied_all
True=0, False=2, eps=1e-09
========== Summary ============
Name: Maximum-Likelihood(False)
MSE of Empirical Distributions: NG
Consistency: OK
MSE of estimators: OK
Physicality Violation: NG
-==============================

Case 6: Least Squares(True)
[OK] N=100
|MSE_EmpiDist - MSE_Analytical| = 0.0011178951319811142
eps = 0.04157787873376893
[OK] N=1000
|MSE_EmpiDist - MSE_Analytical| = 0.0008141913956520553
eps = 0.0013959941188893912



100%|██████████| 2/2 [00:00<00:00, 5862.06it/s]

[OK] Consistency Check
result=4.8927450123350675e-14
eps=1e-10
result < eps: True
[OK] N=100 is_eq_constraint_satisfied_all
True=2, False=0, eps=1e-13
[OK] N=1000 is_eq_constraint_satisfied_all
True=2, False=0, eps=1e-13
[NG] N=100 is_ineq_constraint_satisfied_all
True=1, False=1, eps=1e-09
[NG] N=1000 is_ineq_constraint_satisfied_all
True=0, False=2, eps=1e-09
========== Summary ============
Name: Least Squares(True)
MSE of Empirical Distributions: OK
Consistency: OK
MSE of estimators: OK
Physicality Violation: NG
-==============================

Case 7: Least Squares(False)
[OK] N=100
|MSE_EmpiDist - MSE_Analytical| = 0.003197833620556971
eps = 0.0056941992048731155
[OK] N=1000
|MSE_EmpiDist - MSE_Analytical| = 0.0003667354283614675
eps = 0.003510475985074312


[OK] Consistency Check
result=1.8110893451293842e-14
eps=1e-10
result < eps: True
[OK] N=100 is_eq_constraint_satisfied_all
True=2, False=0, eps=1e-13
[OK] N=1000 is_eq_constraint_satisfied_all
True=2, False=0, eps=1e-13
[NG] N=100 is_ineq_constraint_satisfied_all
True=1, False=1, eps=1e-09
[NG] N=1000 is_ineq_constraint_satisfied_all
True=0, False=2, eps=1e-09
========== Summary ============
Name: Least Squares(False)
MSE of Empirical Distributions: OK
Consistency: OK
MSE of estimators: OK
Physicality Violation: NG
-==============================

